In [55]:
import torchvision.models as models 
import torch.nn.functional as F
from torch import nn
import torch
resnet18 = models.resnet18(pretrained=True) 
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [56]:
resnet18.fc.in_features

512

In [57]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

In [58]:
class Resnt18Rnn(nn.Module):
    def __init__(self, params_model):
        super(Resnt18Rnn, self).__init__()
        
        num_classes = params_model["num_classes"]
        dr_rate= params_model["dr_rate"]
        pretrained = params_model["pretrained"]
        rnn_hidden_size = params_model["rnn_hidden_size"]
        rnn_num_layers = params_model["rnn_num_layers"]
        self.num_classes = 9

        baseModel = models.resnet18(pretrained=pretrained)
        num_features = baseModel.fc.in_features
        baseModel.fc = Identity()
        self.baseModel = baseModel
        self.dropout= nn.Dropout(dr_rate)
        self.rnn = nn.LSTM(num_features, rnn_hidden_size, rnn_num_layers)
        self.fc1 = nn.Linear(rnn_hidden_size, num_classes)
        
    def forward(self, x):
        
        b_z, ts, c, h, w = x.shape
        out_arr = torch.empty(0, b_z, self.num_classes)
        ii = 0
        y = self.baseModel((x[:,ii]))
        out, (hn, cn) = self.rnn(y.unsqueeze(1))
        
        out = self.dropout(out[:,-1])
        out = self.fc1(out)
        
        out_arr = torch.cat(( out_arr, out.view(1, b_z, self.num_classes) ))
        
        for ii in range(1, ts):
            y = self.baseModel((x[:,ii]))
            out, (hn, cn) = self.rnn(y.unsqueeze(1), (hn, cn))
            out = self.dropout(out[:,-1])
            out = self.fc1(out)
            out_arr = torch.cat(( out_arr, out.view(1, b_z, self.num_classes) ))
            
        return out_arr

In [291]:
from torch import nn
class MyConvNet(nn.Module):
    def __init__(self, params_model):
        super(MyConvNet, self).__init__()
        num_classes = params_model["num_classes"]
        dr_rate= params_model["dr_rate"]
        pretrained = params_model["pretrained"]
        rnn_hidden_size = params_model["rnn_hidden_size"]
        rnn_num_layers = params_model["rnn_num_layers"]
        self.num_classes = 9

        self.conv1 = nn.Conv2d(1, 100, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(100, 16, 5)
        
        self.rnn = nn.LSTM(7744, rnn_hidden_size, rnn_num_layers)
        self.fc1 = nn.Linear(rnn_hidden_size, num_classes)
        
    def forward(self, x):
        b_z, ts, c, h, w = x.shape
        out_arr = torch.empty(0, b_z, self.num_classes)
        ii = 0
        x = self.pool(F.relu(self.conv1(x[:,ii])))
        x = self.pool(F.relu(self.conv2(x))) 
        out, (hn, cn) = self.rnn(x.unsqueeze(1))
    
        out = self.fc1(out)
        
        out_arr = torch.cat(( out_arr, out.view(1, b_z, self.num_classes) ))
        
        for ii in range(1, ts):
            print("x:", x.shape)
            print("x[:,ii]:", x[:,ii].shape)
            x = self.pool(F.relu(self.conv1(x[:,ii])))
            x = self.pool(F.relu(self.conv2(x)))
            # x = torch.flatten(x, 1) # flatten all dimensions except batch
            print("x:", x.shape)
            print("x.unsqueeze(1):", x.unsqueeze(1).shape)
            out, (hn, cn) = self.rnn(x.unsqueeze(1), (hn, cn))
            
            out = self.fc1(out)
            out_arr = torch.cat(( out_arr, out.view(1, b_z, self.num_classes) ))
            
        return out_arr

In [59]:
from torchvision import models
from torch import nn

params_model={
    "num_classes": 9,
    "dr_rate": 0.1,
    "pretrained" : True,
    "rnn_num_layers": 1,
    "rnn_hidden_size": 100,}
model = Resnt18Rnn(params_model)

In [60]:
print(model)

Resnt18Rnn(
  (baseModel): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [61]:
h = 100
w = 100

In [62]:
x = torch.randn(3, 3, 1, h, w)

In [63]:
predicted = model(x)

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[3, 1, 100, 100] to have 3 channels, but got 1 channels instead

In [193]:
predicted.shape

torch.Size([20, 3, 9])

In [133]:
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5
inputs

[tensor([[-1.1451, -1.1911, -1.4129]]),
 tensor([[ 0.1971, -1.2499,  1.6056]]),
 tensor([[-0.8239,  0.4378, -1.6552]]),
 tensor([[-1.2103,  0.1556, -0.0288]]),
 tensor([[-0.7368,  1.3944, -1.7665]])]

In [104]:
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    print(i.view(1, 1, -1))

tensor([[[0.2109, 0.8675, 1.9278]]])
tensor([[[ 0.3517, -2.1753, -0.1484]]])
tensor([[[ 1.9604,  0.8455, -2.1195]]])
tensor([[[-1.2150, -0.3774, -0.3942]]])
tensor([[[-1.1273,  1.0420, -0.7233]]])


In [105]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
print(inputs)

tensor([[[ 0.2109,  0.8675,  1.9278]],

        [[ 0.3517, -2.1753, -0.1484]],

        [[ 1.9604,  0.8455, -2.1195]],

        [[-1.2150, -0.3774, -0.3942]],

        [[-1.1273,  1.0420, -0.7233]]])


In [106]:
inputs.shape

torch.Size([5, 1, 3])

In [89]:
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))
hidden

(tensor([[[-0.9437, -1.4374, -0.3225]]]),
 tensor([[[ 0.0283, -0.1561,  0.5289]]]))

In [56]:
predicted.shape

torch.Size([3, 9])

In [178]:
out_arr = torch.empty(0, 120, 9)

In [182]:
output = torch.rand(1, 120, 9)

In [183]:
out_arr = torch.cat(( out_arr, output))

In [185]:
out_arr.shape

torch.Size([1, 120, 9])

In [239]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [242]:
input = torch.randn(120, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0228,  0.0083, -0.1107,  ...,  0.1821, -0.0460,  0.0142],
        [-0.0215,  0.0109, -0.0920,  ...,  0.1614, -0.0373,  0.0286],
        [-0.0114,  0.0216, -0.1090,  ...,  0.1611, -0.0628,  0.0135],
        ...,
        [-0.0309,  0.0245, -0.1167,  ...,  0.1834, -0.0444,  0.0113],
        [-0.0255, -0.0057, -0.1103,  ...,  0.1974, -0.0472,  0.0171],
        [-0.0131,  0.0089, -0.1085,  ...,  0.1854, -0.0577,  0.0180]],
       grad_fn=<AddmmBackward>)


In [ ]:
(3, 120, 1, 32, 32)